In [ ]:
import React, { useState } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery } from '@tanstack/react-query';
import { motion, AnimatePresence } from 'framer-motion';
import { format, subDays, startOfDay, endOfDay, isWithinInterval } from 'date-fns';
import { Calendar as CalendarIcon, Filter, X, Flame } from 'lucide-react';
import { Button } from "@/components/ui/button";
import { Calendar } from "@/components/ui/calendar";
import {
  Popover,
  PopoverContent,
  PopoverTrigger,
} from "@/components/ui/popover";
import {
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
} from "@/components/ui/select";
import {
  Sheet,
  SheetContent,
  SheetHeader,
  SheetTitle,
} from "@/components/ui/sheet";

import MealHistory from '../components/dashboard/MealHistory';
import ScanResult from '../components/scanner/ScanResult';

export default function HistoryPage() {
  const [dateRange, setDateRange] = useState({
    from: subDays(new Date(), 7),
    to: new Date(),
  });
  const [mealTypeFilter, setMealTypeFilter] = useState('all');
  const [selectedMeal, setSelectedMeal] = useState(null);

  const { data: scans = [], isLoading } = useQuery({
    queryKey: ['foodScans'],
    queryFn: () => base44.entities.FoodScan.list('-created_date', 500),
  });

  const { data: userProfiles = [] } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles[0];

  const filteredScans = (scans || []).filter(scan => {
    const scanDate = new Date(scan.created_date);
    const inDateRange = isWithinInterval(scanDate, {
      start: startOfDay(dateRange.from),
      end: endOfDay(dateRange.to),
    });
    const matchesMealType = mealTypeFilter === 'all' || scan.meal_type === mealTypeFilter;
    return inDateRange && matchesMealType;
  });

  const stats = (filteredScans || []).reduce((acc, scan) => ({
    totalCalories: acc.totalCalories + (scan.calories || 0),
    totalProtein: acc.totalProtein + (scan.protein || 0),
    totalCarbs: acc.totalCarbs + (scan.carbs || 0),
    totalFats: acc.totalFats + (scan.fats || 0),
    avgHealthScore: acc.avgHealthScore + (scan.health_score || 0),
    count: acc.count + 1,
  }), { totalCalories: 0, totalProtein: 0, totalCarbs: 0, totalFats: 0, avgHealthScore: 0, count: 0 });

  if (stats.count > 0) {
    stats.avgHealthScore = Math.round(stats.avgHealthScore / stats.count * 10) / 10;
  }

  if (isLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 pb-24">
      <div className="max-w-2xl mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="mb-6"
        >
          <h1 className="text-2xl font-bold text-slate-800">Meal History</h1>
          <p className="text-slate-500 text-sm mt-1">Review your nutrition journey</p>
        </motion.div>

        {/* Filters */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.1 }}
          className="flex flex-wrap gap-3 mb-6"
        >
          <Popover>
            <PopoverTrigger asChild>
              <Button variant="outline" className="rounded-2xl h-11">
                <CalendarIcon className="w-4 h-4 mr-2" />
                {format(dateRange.from, 'MMM d')} - {format(dateRange.to, 'MMM d')}
              </Button>
            </PopoverTrigger>
            <PopoverContent className="w-auto p-0" align="start">
              <Calendar
                mode="range"
                selected={dateRange}
                onSelect={(range) => range && setDateRange(range)}
                numberOfMonths={1}
              />
            </PopoverContent>
          </Popover>

          <Select value={mealTypeFilter} onValueChange={setMealTypeFilter}>
            <SelectTrigger className="w-40 rounded-2xl h-11">
              <Filter className="w-4 h-4 mr-2" />
              <SelectValue placeholder="Meal type" />
            </SelectTrigger>
            <SelectContent>
              <SelectItem value="all">All Meals</SelectItem>
              <SelectItem value="breakfast">🌅 Breakfast</SelectItem>
              <SelectItem value="lunch">☀️ Lunch</SelectItem>
              <SelectItem value="dinner">🌙 Dinner</SelectItem>
              <SelectItem value="snack">🍎 Snack</SelectItem>
            </SelectContent>
          </Select>
        </motion.div>

        {/* Period Stats */}
        {filteredScans.length > 0 && (
          <motion.div
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.2 }}
            className="bg-white rounded-3xl p-6 border border-slate-100 shadow-sm mb-6"
          >
            <h3 className="text-sm font-medium text-slate-500 mb-4">Period Summary</h3>
            <div className="grid grid-cols-2 md:grid-cols-4 gap-4">
              <div className="text-center p-3 bg-orange-50 rounded-2xl">
                <Flame className="w-5 h-5 text-orange-500 mx-auto mb-1" />
                <p className="text-xl font-bold text-slate-800">{Math.round(stats.totalCalories).toLocaleString()}</p>
                <p className="text-xs text-slate-500">Total kcal</p>
              </div>
              <div className="text-center p-3 bg-rose-50 rounded-2xl">
                <p className="text-xl font-bold text-slate-800">{Math.round(stats.totalProtein)}g</p>
                <p className="text-xs text-slate-500">Protein</p>
              </div>
              <div className="text-center p-3 bg-amber-50 rounded-2xl">
                <p className="text-xl font-bold text-slate-800">{Math.round(stats.totalCarbs)}g</p>
                <p className="text-xs text-slate-500">Carbs</p>
              </div>
              <div className="text-center p-3 bg-emerald-50 rounded-2xl">
                <p className="text-xl font-bold text-slate-800">{stats.avgHealthScore}</p>
                <p className="text-xs text-slate-500">Avg Score</p>
              </div>
            </div>
          </motion.div>
        )}

        {/* Meal List */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.3 }}
        >
          <MealHistory 
            scans={filteredScans} 
            onSelectMeal={setSelectedMeal}
          />
        </motion.div>

        {/* Meal Detail Sheet */}
        <Sheet open={!!selectedMeal} onOpenChange={() => setSelectedMeal(null)}>
          <SheetContent side="bottom" className="h-[90vh] rounded-t-3xl">
            <SheetHeader className="mb-4">
              <div className="flex items-center justify-between">
                <SheetTitle>Meal Details</SheetTitle>
                <Button
                  variant="ghost"
                  size="icon"
                  onClick={() => setSelectedMeal(null)}
                  className="rounded-full"
                >
                  <X className="w-5 h-5" />
                </Button>
              </div>
            </SheetHeader>
            {selectedMeal && (
              <div className="overflow-y-auto max-h-[calc(90vh-80px)]">
                <ScanResult
                  result={selectedMeal}
                  onAddToLog={() => {}}
                  onScanAgain={() => setSelectedMeal(null)}
                  userAllergies={userProfile?.allergies || []}
                />
              </div>
            )}
          </SheetContent>
        </Sheet>
      </div>
    </div>
  );
}